In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

ImportError: No module named sklearn.model_selection

In [5]:
path = '../data/adult/'

In [ ]:
train_file = path + 'adult.data'
test_file = path + 'adult.test'

In [ ]:
def raw_to_df(fname):
  return pd.read_csv(fname, header=None, delimiter = ' *, *', engine='python',
                     names=['age', 'workclass', 'fnlwgt', 'education', 'education-num',
                            'marital-status', 'occupation', 'relationship', 'race', 'sex',
                            'capital-gain', 'capital-loss', 'hours-per-week',
                            'native-country', 'income'])

df_train_init = raw_to_df(train_file)
df_test = raw_to_df(test_file)

In [ ]:
#separate original train into train and validation
df_train, df_valid = train_test_split(df_train_init, test_size=0.05)

In [ ]:
print(df_train.shape)
print(df.valid.shape)
print(df_test.shape)

In [ ]:
df_train[(df_train == '?').any(axis=1)]

In [ ]:
age_buckets = [0, 18, 25, 30, 35, 40, 45, 50, 55, 60, 65, float('+inf')]
continuous_cols = ['capital-gain', 'capital-loss', 'education-num', 'hours-per-week']

def featurize(df):
  df = df.copy()
  # income to binary classification
  df['income'].replace(['<=50K', '<=50K.'], 0, inplace=True)
  df['income'].replace(['>50K', '>50K.'], 1, inplace=True)
  # prune columns
  df.drop('fnlwgt', axis=1, inplace=True)
  # bucket age
  df['age'] = pd.cut(df['age'], age_buckets, right=False)
  # one-hot
  df = pd.get_dummies(df, sparse=True)
  # split X, y
  X = df.drop('income', axis=1)
  y = df['income']
  # normalize continuous cols in X
  for col in continuous_cols:
    X[col] = (X[col] - X[col].mean()) / X[col].std()
  return X, y

In [ ]:
X_train, y_train = featurize(df_train)
X_valid, y_valid = featurize(df_valid)
X_test, y_test = featurize(df_test)

In [ ]:
# make sure train and test are not missing features
def match_cols(a, b):
  b_missing = set(a.columns) - set(b.columns)
  a_missing = set(b.columns) - set(a.columns)
  for col in a_missing:
    a[col] = 0
  for col in b_missing:
    b[col] = 0

match_cols(X_train, X_test)
match_cols(X_train, X_valid)
print(X_train.shape)
print(X_test.shape)
print(X_valid.shape)

In [ ]:
print(df_train['race'].unique())
print(df_train['sex'].unique())
nums = [str(i) for i in range(11)]
print(nums)
df_train['age'] = pd.cut(df_train['age'], age_buckets, right=False, labels=range(11))
df_valid['age'] = pd.cut(df_valid['age'], age_buckets, right=False, labels=range(11))
df_test['age'] = pd.cut(df_test['age'], age_buckets, right=False, labels=range(11))
print(df_train['age'].unique())

In [ ]:
race_ids = {'White': 0, 'Black': 1, 'Asian-Pac-Islander': 2, 'Amer-Indian-Eskimo': 3, 'Other': 4}
sex_ids = {'Male': 0, 'Female': 1}

def featurize_protected(df, protected, id_map):
  col = df[protected].copy()
  col.replace(id_map, inplace=True)
  return col

race_train = featurize_protected(df_train, 'race', race_ids)
race_valid = featurize_protected(df_valid, 'race', race_ids)
race_test = featurize_protected(df_test, 'race', race_ids)
sex_train = featurize_protected(df_train, 'sex', sex_ids)
sex_valid = featurize_protected(df_valid, 'sex', sex_ids)
sex_test = featurize_protected(df_test, 'sex', sex_ids)
age_train = df_train['age'].copy()
age_valid = df_valid['age'].copy()
age_test = df_test['age'].copy()

In [145]:
X_train.to_pickle(path + 'X_train.pkl')
y_train.to_pickle(path + 'y_train.pkl')

X_valid.to_pickle(path + 'X_valid.pkl')
y_valid.to_pickle(path + 'y_valid.pkl')

X_test.to_pickle(path + 'X_test.pkl')
y_test.to_pickle(path + 'y_test.pkl')

race_train.to_pickle(path + 'race_train.pkl')
race_valid.to_pickle(path + 'race_valid.pkl')
race_test.to_pickle(path + 'race_test.pkl')

sex_train.to_pickle(path + 'sex_train.pkl')
sex_valid.to_pickle(path + 'sex_valid.pkl')
sex_test.to_pickle(path + 'sex_test.pkl')

age_train.to_pickle(path + 'age_train.pkl')
age_valid.to_pickle(path + 'age_valid.pkl')
age_test.to_pickle(path + 'age_test.pkl')

In [149]:
# NOW COMPUTE STATS ON TRAIN AND TEST -- OUT OF DATE AS NOT COMPUTED ON VALID

num_train = sum(1 for i,j in sex_train.iteritems())
num_test = sum(1 for i,j in sex_test.iteritems())

print ("TRAIN prop women: ", 100.0*sum(1 for i, j in sex_train.iteritems() if (j == 1))/num_train)
print ("TRAIN prop men: ", 100.0*sum(1 for i, j in sex_train.iteritems() if (j == 0))/num_train)

print ("TEST prop women: ", 100.0*sum(1 for i, j in sex_test.iteritems() if (j == 1))/num_test)
print ("TEST prop men: ", 100.0*sum(1 for i, j in sex_test.iteritems() if (j == 0))/num_test)

print ("Total pop women: ", 100.0*(sum(1 for i, j in sex_train.iteritems() if (j == 1)) + sum(1 for i, j in sex_test.iteritems() if (j == 1)))/(num_train+num_test))
print ("Total pop men: ", 100.0*(sum(1 for i, j in sex_train.iteritems() if (j == 0)) + sum(1 for i, j in sex_test.iteritems() if (j == 0)))/(num_train+num_test))

('TRAIN prop women: ', 33.07945087681582)
('TRAIN prop men: ', 66.92054912318417)
('TEST prop women: ', 33.29648056016215)
('TEST prop men: ', 66.70351943983785)
('Total pop women: ', 33.15179558576635)
('Total pop men: ', 66.84820441423365)


In [150]:
num_train = sum(1 for i,j in race_train.iteritems())
num_test = sum(1 for i,j in race_test.iteritems())

for q in range(5):
    print ("Total race pop demographic ", q, ": ", 100.0*(sum(1 for i, j in race_train.iteritems() if (j == q)) + sum(1 for i, j in race_test.iteritems() if (j == q)))/(num_train+num_test))


('Total race pop demographic ', 0, ': ', 85.5042791040498)
('Total race pop demographic ', 1, ': ', 9.592154293435977)
('Total race pop demographic ', 2, ': ', 3.110028254371238)
('Total race pop demographic ', 3, ': ', 0.9622865566520618)
('Total race pop demographic ', 4, ': ', 0.83125179149093)


In [151]:
num_train = sum(1 for i,j in age_train.iteritems())
num_test = sum(1 for i,j in age_test.iteritems())
s = 0
for q in range(11):
    s += 100.0*(sum(1 for i, j in age_train.iteritems() if (j == q)) + sum(1 for i, j in age_test.iteritems() if (j == q)))/(num_train+num_test)
    print ("Total age pop demographic ", q, ": ", 100.0*(sum(1 for i, j in age_train.iteritems() if (j == q)) + sum(1 for i, j in age_test.iteritems() if (j == q)))/(num_train+num_test))
print(s)

('Total age pop demographic ', 0, ': ', 1.2182138323573972)
('Total age pop demographic ', 1, ': ', 16.04561647762172)
('Total age pop demographic ', 2, ': ', 12.45444494492445)
('Total age pop demographic ', 3, ': ', 13.295933827443594)
('Total age pop demographic ', 4, ': ', 13.175136153310675)
('Total age pop demographic ', 5, ': ', 11.789034028090578)
('Total age pop demographic ', 6, ': ', 10.167478809221572)
('Total age pop demographic ', 7, ': ', 7.790426272470415)
('Total age pop demographic ', 8, ': ', 5.761434830678515)
('Total age pop demographic ', 9, ': ', 4.029319028704803)
('Total age pop demographic ', 10, ': ', 4.272961795176283)
100.0
